In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [ ]:
# Upload your CSV from local
from google.colab import files
uploaded = files.upload()

# Load it into a DataFrame
df = pd.read_excel("survey.xlsx")  # replace with actual filename
df.head()



In [ ]:
# Upload your CSV from local
from google.colab import files
uploaded = files.upload()

# Load it into a DataFrame
df_pitanja = pd.read_excel("question_sheet.xlsx")  # replace with actual filename

Saving matrei_pitanja.xlsx to matrei_pitanja (9).xlsx


In [ ]:
df_pitanja.head()

In [ ]:
ordinal_vars = []
for qid, qtype in zip(df_pitanja["QuestionID"], df_pitanja["Type"]):
    if qtype == "Scale":
        ordinal_vars.append(str(qid))


In [ ]:
ordinal_vars = [str(int(float(col))) if str(col).endswith('.0') else str(col) for col in ordinal_vars]

In [ ]:
select_one = []
for qid, qtype in zip(df_pitanja["QuestionID"], df_pitanja["Type"]):
    if qtype == "SelectOne":
        select_one.append(qid)

In [ ]:
select_one = [str(int(float(col))) if str(col).endswith('.0') else str(col) for col in select_one]

In [ ]:
multi_nominal_cols = []
for qid, qtype in zip(df_pitanja["QuestionID"], df_pitanja["Type"]):
    if qtype == "SelectMultiple":
        multi_nominal_cols.append(qid)

In [ ]:
multi_nominal_cols = [str(int(float(col))) if str(col).endswith('.0') else str(col) for col in multi_nominal_cols]

In [ ]:
# See missing values per column
df_mising=df.isnull().sum().sort_values(ascending=False)

In [ ]:
za_drop = []

for index, value in df_mising.items():
    if value > 8:
        za_drop.append(index)

In [ ]:
za_drop = [str(int(float(col))) if str(col).endswith('.0') else str(col) for col in za_drop]

In [ ]:
za_drop2 = []

for qid, qtype in zip(df_pitanja["QuestionID"], df_pitanja["Type"]):
    if qtype == "Text":
        za_drop2.append(str(qid))  # convert to string here


In [ ]:
za_drop2_cleaned = [str(int(float(col))) if str(col).endswith('.0') else str(col) for col in za_drop2]

In [ ]:
# 🔹 Step 2: Drop rows where 379.1 is missing
df = df[df['379'].notna()]

# 🔹 Step 3: Convert 379.1 to binary target variable
df['379_binary'] = df['379'].apply(lambda x: 1 if x <= 4 else 0)
df = df.drop(columns=['379'])

In [ ]:
df = df.drop(columns=za_drop)

In [ ]:
df = df.drop(columns=za_drop2_cleaned, errors='ignore')


In [ ]:
df = df.iloc[:, 10:]


In [ ]:
df = df.iloc[:, 4:]

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Step 2: Filter out columns that don’t exist in the DataFrame
ordinal_vars = [col for col in ordinal_vars if col in df.columns]

# Step 3: Convert only valid ones
for col in ordinal_vars:
    df[col] = df[col].astype('Int64')


In [ ]:
for col in multi_nominal_cols:
    if col in df.columns:
        # Step 1: Clean missing and convert to string
        df[col] = df[col].fillna('').astype(str)

        # Step 2: One-hot encode selections
        dummies = df[col].str.get_dummies(sep='/')
        dummies.columns = [f"{col}_{c.strip()}" for c in dummies.columns]

        # Step 3: Create missing-all flag BEFORE dropping or joining
        df[f'{col}_missing_all'] = dummies.sum(axis=1).eq(0).astype(int)

        # Step 4: Replace original with dummies
        df = df.drop(columns=[col])
        df = pd.concat([df, dummies], axis=1)


In [ ]:
for col in df.select_dtypes(include='number').columns:
    median = df[col].median()
    if pd.api.types.is_integer_dtype(df[col]):
        median = int(round(median))  # force integer
    df[col] = df[col].fillna(median)


In [ ]:
for col in select_one:
    if col in df.columns:
        # Step 1: Clean missing and convert to string
        df[col] = df[col].fillna('').astype(str)

        # Step 2: One-hot encode selections
        dummies = df[col].str.get_dummies(sep=',')
        dummies.columns = [f"{col}_{c.strip()}" for c in dummies.columns]

        # Step 3: Create missing-all flag BEFORE dropping or joining
        df[f'{col}_missing_all'] = dummies.sum(axis=1).eq(0).astype(int)

        # Step 4: Replace original with dummies
        df = df.drop(columns=[col])
        df = pd.concat([df, dummies], axis=1)


In [ ]:
output_path = ".csv"
df.to_csv(output_path, index=False)

# Step 7: Trigger download (Colab only)
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>